In [1]:
from pyqtorch.matrices import single_Z, ZZ
from pyqtorch.ansatz import AlternateLayerAnsatz, OneLayerXRotation, OneLayerZRotation, OneLayerEntanglingAnsatz
from pyqtorch.core.circuit import QuantumCircuit

/home/aag/anaconda3/envs/pynn/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [50]:
import torch as th
import numpy as np
import copy

we start initialising the `QuantumCircuit` instance in order to observe the typical shape of an input/output state in the PyQ format

In [81]:
N = 4
qc = QuantumCircuit(N)
psi = qc.uniform_state(1)
psi_0 = copy.deepcopy(psi)
psi_0.shape

torch.Size([2, 2, 2, 2, 1])

we perform a deepcopy of `psi` as some operations below (e.g. `hamiltonian evolution`) will overwrite it

In [82]:
def overlap(state1, state2):
    N = len(state1.shape)-1
    state1_T = th.transpose(state1, N, 0)
    overlap = th.tensordot(state1.T.conj(), state2, dims=N)
    return float(th.abs(overlap**2).flatten())

In [83]:
print("Initial overlap: ", overlap(psi_0, psi_0))

Initial overlap:  1.0


## Hamiltonian Evolution

In [54]:
from pyqtorch.core.operation import hamiltonian_evolution

now let us define a simple Hamiltonian for the 2-qubits system, like a $\sigma_Z \otimes \sigma_Z$, in dense format as a $(N^2, N^2)$ tensor

In [55]:
sigmaz = th.diag(th.tensor([1.0, -1.0], dtype=th.cdouble))
H = th.kron(sigmaz, sigmaz)
H.shape
H

tensor([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j, -1.+0.j,  0.+0.j, -0.+0.j],
        [ 0.+0.j,  0.+0.j, -1.+0.j, -0.+0.j],
        [ 0.+0.j, -0.+0.j, -0.+0.j,  1.-0.j]], dtype=torch.complex128)

The overlap with itself should stay 1 after evolving for $t=0$, let's check this

In [56]:
t_evo = th.tensor([0], dtype=th.cdouble)
psi = hamiltonian_evolution(H,
                    psi, t_evo,
                    range(N), N)

In [57]:
print(f"Overlap after {t_evo} : ", overlap(psi, psi_0))

Overlap after tensor([0.+0.j], dtype=torch.complex128) :  1.0


In [58]:
t_evo = th.tensor([th.pi/4], dtype=th.cdouble)
psi = hamiltonian_evolution(H,
                    psi, t_evo,
                    range(N), N
    )

In [59]:
print(f"Overlap after {t_evo} : ", overlap(psi, psi_0))

Overlap after tensor([0.7854+0.j], dtype=torch.complex128) :  0.50000000002474


## Arbitrary rotation on the Bloch sphere

In [84]:
from pyqtorch.core.operation import U

In [91]:
psi_rot = U(phi=th.tensor([th.pi/2]), theta=th.tensor([th.pi]), omega=th.tensor([0.]),
            state=qc.uniform_state(1), qubits=[0], N_qubits=N)